In [1]:
import telepot
import bs4
import requests
import youtube_dl

In [2]:
bot = telepot.Bot('api_key')

In [3]:
bot.getMe()

In [4]:
def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'progress_hooks': [my_hook],
}

In [5]:
base_url = "https://www.youtube.com/results?search_query="
def handle(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)
    print(content_type, chat_type, chat_id)

    if content_type == 'text':
        if msg['text'] != "/start":
            print("received request from:" + str(chat_id))
            bot.sendMessage(chat_id, msg['text']+" is being downloaded...")
            song_name = msg['text']
            search_query = song_name.replace(" ", '+')+"+full+audio"
            url = base_url+search_query
#             print(url)
            r = requests.get(url)
            soup = bs4.BeautifulSoup(r.text,'lxml')
            z = soup.select("div > a")
#             print("2: "+z[2]['href'])
#             print("3: "+z[3]['href'])
#             print("4: "+z[4]['href'])
#             print("5: "+z[5]['href'])
            if ("full+audio" in z[2]['href']):
                if ("watch" in z[3]['href']):
                    song_url = "https://www.youtube.com"+z[3]['href']
                else:
                    song_url = "https://www.youtube.com"+z[4]['href']
            else:
                song_url = "https://www.youtube.com"+z[2]['href']
            song_name = str(chat_id) + "_" + msg['text'] + "_.mp3"
            print("song_url: "+song_url)
            ydl_opts['outtmpl'] = str(chat_id) + "_" + msg['text'] + '_.%(ext)s'
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download([song_url])
            print("Song downloaded...")
            audio = open(song_name,'rb')
            bot.sendMessage(chat_id, "Download Complete....Sending you the song")
            print("Sending...")
            bot.sendAudio(chat_id, audio,title=ydl.extract_info(song_url, download=False)['title'])
            print("Sent...!!!")
        else:
            bot.sendMessage(chat_id, "You can now send names of song you want to download...")
            

In [6]:
bot.message_loop(handle)